In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# CSVファイルを読み込む
df = pd.read_csv("/home/eiji/project/g/data/TON_iot/Processed_IoT_dataset/IoT_Garage_Door.csv")
print(df.isna().sum())
df = df.dropna()
print(df.isna().sum())
df

# unique_values = df['sphone_signal'].unique()
# print(unique_values)


/tmp/ipykernel_66166/523609548.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/eiji/project/g/data/TON_iot/Processed_IoT_dataset/IoT_Garage_Door.csv")


date             21335
time             21335
door_state       50269
sphone_signal    50269
label                0
type                 0
dtype: int64
date             0
time             0
door_state       0
sphone_signal    0
label            0
type             0
dtype: int64


,date,time,door_state,sphone_signal,label,type
0,1-Apr-19,20:53:44,open,true,0,normal
1,1-Apr-19,20:53:49,closed,false,0,normal
2,1-Apr-19,20:53:49,open,true,0,normal
3,1-Apr-19,20:53:54,closed,false,0,normal
4,1-Apr-19,20:53:54,open,true,0,normal
...,...,...,...,...,...,...
591441,27-Apr-19,11:03:21,closed,0.0,0,normal
591442,27-Apr-19,11:03:21,closed,0.0,0,normal
591443,27-Apr-19,11:03:21,closed,0.0,0,normal
591444,27-Apr-19,11:03:22,closed,0.0,0,normal


In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# 全ての値を文字列として扱い、余分なスペースを削除
df['sphone_signal'] = df['sphone_signal'].astype(str).str.strip()
# 値の置換
df['sphone_signal'] = df['sphone_signal'].replace({'true': 1, 'false': 0, '0': 0, '1': 1, '0.0': 0, '1.0': 1})
# door_stateは文字列として扱い、sphone_signalはそのまま数値として残す
df['door_state'] = df['door_state'].astype(str)

# OneHotEncoderの使用
encoder = OneHotEncoder(sparse=False, dtype=int)
# door_stateにOneHotエンコーディングを適用
encoded_columns = encoder.fit_transform(df[['door_state', 'sphone_signal']])
# 新しいエンコードされたカラム名を作成
encoded_col_names = encoder.get_feature_names_out(['door_state', 'sphone_signal'])

# エンコードされたデータをデータフレームに結合する前にインデックスをリセット
df.reset_index(drop=True, inplace=True)
df_encoded = pd.DataFrame(encoded_columns, columns=encoded_col_names, index=df.index)

# エンコードされたデータをデータフレームに結合
df = pd.concat([df, df_encoded], axis=1)

# 元のカラムを削除
df.drop(['door_state'], axis=1, inplace=True)
df


/tmp/ipykernel_66166/1643927704.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sphone_signal'] = df['sphone_signal'].replace({'true': 1, 'false': 0, '0': 0, '1': 1, '0.0': 0, '1.0': 1})
/home/eiji/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,date,time,sphone_signal,label,type,door_state_closed,door_state_open,sphone_signal_0,sphone_signal_1
0,1-Apr-19,20:53:44,1,0,normal,0,1,0,1
1,1-Apr-19,20:53:49,0,0,normal,1,0,1,0
2,1-Apr-19,20:53:49,1,0,normal,0,1,0,1
3,1-Apr-19,20:53:54,0,0,normal,1,0,1,0
4,1-Apr-19,20:53:54,1,0,normal,0,1,0,1
...,...,...,...,...,...,...,...,...,...
541172,27-Apr-19,11:03:21,0,0,normal,1,0,1,0
541173,27-Apr-19,11:03:21,0,0,normal,1,0,1,0
541174,27-Apr-19,11:03:21,0,0,normal,1,0,1,0
541175,27-Apr-19,11:03:22,0,0,normal,1,0,1,0


# 時間

In [3]:
df['time'] = df['time'].str.strip()

In [4]:
df['time']  = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time

In [5]:
df['hour'] = df['time'].apply(lambda x: x.hour)
df['minute'] = df['time'].apply(lambda x: x.minute)
df['second'] = df['time'].apply(lambda x: x.second)

In [6]:
new_order = ['hour', 'minute', 'second', 'sphone_signal_0', 'sphone_signal_1', 'door_state_open', 'door_state_closed', 'label']

In [7]:
df = df[new_order]

In [8]:
df

,hour,minute,second,sphone_signal_0,sphone_signal_1,door_state_open,door_state_closed,label
0,20,53,44,0,1,1,0,0
1,20,53,49,1,0,0,1,0
2,20,53,49,0,1,1,0,0
3,20,53,54,1,0,0,1,0
4,20,53,54,0,1,1,0,0
...,...,...,...,...,...,...,...,...
541172,11,3,21,1,0,0,1,0
541173,11,3,21,1,0,0,1,0
541174,11,3,21,1,0,0,1,0
541175,11,3,22,1,0,0,1,0


In [9]:
df.to_csv('./preprocessed_data/GarageDoor_GD.csv', index=False)